In [13]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import pretty_midi
import librosa
import librosa.display
import gc
from sklearn.preprocessing import StandardScaler
import warnings
from collections import Counter
from torch.utils.data import Dataset
import torch
from pympler import asizeof
import torch.optim as optim 



import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset


from sklearn.preprocessing import StandardScaler

from Preprocessing import *
#from ExtractGenre import *
from CNN_ExtractGenre import *
from PolyphonicPreprocessing import *
import Util as Util

import DatasetLoader as DL
import Model as M



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
GenreMapping = {'metal': 0, 'disco': 1, 'classical': 2, 'hiphop': 3, 'jazz': 4,
          'country': 5, 'pop': 6, 'blues': 7, 'reggae': 8, 'rock': 9}

In [13]:
Path = os.path.realpath('clean_midi/Eminem/Stan.mid')
mid = mido.MidiFile(Path)
#print(mid)



In [87]:
PolyDataset = PolyphonicPreProcessing(nDir = 1500)

Preprocessing: 100%|██████████| 1500/1500 [06:18<00:00,  3.97it/s]


In [88]:
torch.save(PolyDataset, 'PolyphonicDataset.pt')

# 1 Instrument 

In [63]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('GPU available')
else:
    device = torch.device("cpu")
    print('GPU not available')

    

discriminator = M.Discriminator(cond_1d_size=2, instrument_size=1)
discriminator.apply(M.weights_init)
discriminator.to(device)
# Load the state dict previously saved
discriminator_state_dict = torch.load('discriminator_parameters.torch', map_location=torch.device('cpu'))
# Update the network parameters
discriminator.load_state_dict(discriminator_state_dict)

generator = M.Generator(input_size=100, cond_1d_size=2, instrument_size=1, n_hlayers=128)
generator.apply(M.weights_init)
generator.to(device)
# Load the state dict previously saved
generator_state_dict = torch.load('generator_parameters.torch', map_location=torch.device('cpu'))
# Update the network parameters
generator.load_state_dict(generator_state_dict)


dis_opt = optim.Adam(discriminator.parameters(), lr=2e-4)
# Load the state dict previously saved
dis_opt_state_dict = torch.load('dis_opt_state.torch', map_location=torch.device('cpu'))
# Update the network parameters
dis_opt.load_state_dict(dis_opt_state_dict)

gen_opt = optim.RMSprop(generator.parameters(), lr=1e-5)
# Load the state dict previously saved
gen_opt_state_dict = torch.load('gen_opt_state.torch', map_location=torch.device('cpu'))
# Update the network parameters
gen_opt.load_state_dict(gen_opt_state_dict)

GPU not available


In [82]:
generator.eval()
noise = torch.randn([1, 100], device=device)

Instrument = 'Guitar'
bar = 30

prev_bar = Dataset[Instrument][bar]['Bars'][0].to_dense().float().to(device)
InstrumentCode = Dataset[Instrument][bar]['Program']
Tempo = Dataset[Instrument][bar]['Tempo'][0]
cond_1d = torch.tensor([[Tempo, InstrumentCode]], dtype= torch.float32)
Bars = []
Bars.append(prev_bar)
prev_bar = prev_bar.unsqueeze(0).unsqueeze(0) 


for i in range(8):
   with torch.no_grad():
      generated_bar = generator(noise, prev_bar, cond_1d, 1)

   generated_bar=generated_bar.cpu().numpy()
   binary_bar = (generated_bar.reshape(128, 16) > 0.5).astype(int)

   if i > 1:
      prev_bar = torch.tensor(binary_bar, dtype=torch.float32, device=device).unsqueeze(0).unsqueeze(0)

   Bars.append(binary_bar)



ConcBars = np.concatenate(Bars, axis = 1)

MonoBarsToMIDI(ConcBars, title='test', Instrument=InstrumentCode)